In [1]:
import sys
# 프로젝트 루트 직접 지정
sys.path.insert(0, r"c:\Users\tjrrj\vscode\doritest")

In [2]:
import pandas as pd
import gspread
from google.oauth2 import service_account
from datetime import datetime
from modules import add_surrogate_key


In [3]:
df = pd.read_excel(
    r"D:\Download\coupang_eats_2025-11.xlsx",
    header=1           # 2행을 헤더로 사용
)
df = df[1:]
df = df[["주문번호", "일자", "시간", "주문금액"]].tail()
df

c:\Users\tjrrj\vscode\doritest\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,주문번호,일자,시간,주문금액
351,221F46,2025-11-30,2025-11-30 17:21:23,16100.0
352,2TAWU6,2025-11-30,2025-11-30 17:35:32,28000.0
353,1NPZ3Z,2025-11-30,2025-11-30 17:46:40,15000.0
354,0RWZDN,2025-11-30,2025-11-30 18:44:02,31500.0
355,1VWRZ4,2025-11-30,2025-11-30 20:22:11,20500.0


In [4]:
df["일자"] = pd.to_datetime(df["일자"]).dt.date

In [5]:
df = add_surrogate_key(df, natural_key_cols=["주문번호", "일자", "시간"])

In [6]:
df["주문금액"] = df["주문금액"].astype(int)
df.rename(columns={"key" : "order_id"}, inplace=True)
df

,order_id,주문번호,일자,시간,주문금액
351,3a5bbf466cab4e30,221F46,2025-11-30,2025-11-30 17:21:23,16100
352,80e79b569315f395,2TAWU6,2025-11-30,2025-11-30 17:35:32,28000
353,a5a11b36053c1c07,1NPZ3Z,2025-11-30,2025-11-30 17:46:40,15000
354,1c8a942e16dc2a28,0RWZDN,2025-11-30,2025-11-30 18:44:02,31500
355,27cfd89220369a4c,1VWRZ4,2025-11-30,2025-11-30 20:22:11,20500


In [ ]:
"""
구글시트 함수
- 파일명: plugins/load/gsheet.py
"""
import re
from datetime import datetime
from typing import Optional, Union, List
import pandas as pd
import gspread
from google.oauth2 import service_account


# ===== 기본 설정 =====
# key 파일 경로 및 스프레드시트 ID
DEFAULT_CREDENTIALS = r"C:\Users\tjrrj\vscode\dori\config\glowing-palace-465904-h6-7f82df929812.json"

DEFAULT_SPREADSHEET_ID = "https://docs.google.com/spreadsheets/d/17jloeN0cqpQrPAfcBaINjU4u02gMXgRfGfpvZ7uMj-E/edit?usp=sharing"

DEFAULT_SHEET = "시트1"


def save_to_gsheet(
    df: Optional[pd.DataFrame] = None,
    sheet_name: Optional[str] = None,
    mode: str = "append_unique",
    action: Optional[str] = None,
    primary_key: Union[str, List[str], None] = None,
    credentials_path: Optional[str] = None,
    spreadsheet_id: Optional[str] = None,
):
    """
    
    gsheet(
    df,
    sheet_name="시트1",
    primary_key="order_id",        # ← build_order_keys가 만든 surrogate key
    mode="append_unique"      # ← 중복이면 안 넣고, 신규만 아래에 추가
    )
    
    """
    # 호환성 처리: df에 문자열이 오고 sheet_name이 비어있으면 읽기용 sheet_name으로 해석
    if isinstance(df, str) and sheet_name is None:
        sheet_name = df
        df = None

    # 기본값
    if sheet_name is None:
        sheet_name = DEFAULT_SHEET
    credentials_path = credentials_path or DEFAULT_CREDENTIALS
    spreadsheet_id = spreadsheet_id or DEFAULT_SPREADSHEET_ID

    # URL에서 /d/<key> 추출 (키가 직접 들어오면 그대로 사용)
    if "/d/" in spreadsheet_id:
        m = re.search(r"/d/([a-zA-Z0-9_-]+)", spreadsheet_id)
        if m:
            spreadsheet_id = m.group(1)

    # 인증
    try:
        SCOPES = [
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive",
        ]
        creds = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=SCOPES
        )
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_key(spreadsheet_id)
    except Exception as e:
        print(f"❌ 인증 실패: {str(e)}")
        return None

    # ===== CLEAR =====
    if action == "clear":
        print(f"▶️ 초기화: {sheet_name}")
        try:
            sheet = spreadsheet.worksheet(sheet_name)
            header = sheet.row_values(1)
            sheet.clear()
            if header:
                sheet.update("A1", [header])
            print("✅ 완료")
            return True
        except Exception as e:
            print(f"❌ 실패: {str(e)}")
            return False

    # ===== READ =====
    if df is None:
        print(f"▶️ 읽기: {sheet_name}")
        try:
            sheet = spreadsheet.worksheet(sheet_name)
            result = pd.DataFrame(sheet.get_all_records())
            print(f"✅ 완료: {len(result)}건")
            return result
        except Exception as e:
            print(f"❌ 실패: {str(e)}")
            return pd.DataFrame()

    # ===== UPLOAD =====
    timestamp_column = "uploaded_at"
    print(f"▶️ 업로드: {sheet_name} ({len(df)}건, {mode})")

    try:
        # 시트 열기 (없으면 생성)
        try:
            sheet = spreadsheet.worksheet(sheet_name)
            # 헤더가 비어있으면 업로드 DF 기준으로 세팅
            current_header = sheet.row_values(1)
            if not current_header:
                headers = df.columns.tolist() + [timestamp_column]
                sheet.update("A1", [headers])
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=sheet_name, rows=1000, cols=50)
            headers = df.columns.tolist() + [timestamp_column]
            sheet.update("A1", [headers])
            print(f"   📝 새 시트 생성: {sheet_name}")

        # 원본 보존한 채 타임스탬프 추가
        up_df = df.copy()
        up_df[timestamp_column] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # ===== 덮어쓰기 =====
        if mode == "overwrite":
            sheet.clear()
            data = [up_df.columns.tolist()] + up_df.astype(str).values.tolist()
            sheet.update("A1", data)
            print(f"✅ 완료 (덮어쓰기): {len(up_df)}건")
            return {"success": True, "new": len(up_df)}

        # ===== 단순 추가 =====
        if mode == "append":
            sheet.append_rows(up_df.astype(str).values.tolist())
            print(f"✅ 완료 (추가): {len(up_df)}건")
            return {"success": True, "new": len(up_df)}

        # ===== 중복 제외 추가 (append_unique) =====
        existing_df = pd.DataFrame(sheet.get_all_records())

        # 중복 판정 키 결정
        if primary_key is None:
            key_columns = [c for c in up_df.columns if c != timestamp_column]
        elif isinstance(primary_key, str):
            key_columns = [primary_key]
        else:
            key_columns = list(primary_key)

        if len(existing_df) > 0 and all(col in existing_df.columns for col in key_columns):
            existing_df["_key"] = existing_df[key_columns].astype(str).agg("|".join, axis=1)
            up_df["_key"] = up_df[key_columns].astype(str).agg("|".join, axis=1)
            new_df = up_df[~up_df["_key"].isin(existing_df["_key"])].drop(columns=["_key"])
            dup = len(up_df) - len(new_df)
        else:
            new_df = up_df
            dup = 0

        if len(new_df) > 0:
            sheet.append_rows(new_df.astype(str).values.tolist())

        print(f"✅ 완료: 신규 {len(new_df)}건, 중복 {dup}건")
        return {"success": True, "new": len(new_df), "duplicate": dup}

    except Exception as e:
        print(f"❌ 실패: {str(e)}")
        return {"success": False, "error": str(e)}


In [ ]:
save_to_gsheet(
    df,
    sheet_name="시트1",
    primary_key="order_id",        # ← build_order_keys가 만든 surrogate key
    mode="append_unique"      # ← 중복이면 안 넣고, 신규만 아래에 추가
)

NameError: name 'save_to_gsheet' is not defined